In [ ]:
#@title loko { display-mode: "form" }
seq_len = 10 #@param {type:"number"}
numero_moleculas_geradas = 1000 
modelo = 'Rnn'
dataset = "CheBi" #@param ["CheBi", "CheMBL"]

#Inicio colab e imports

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive
%cd My Drive
%cd geração de moléculas

Mounted at /gdrive
/gdrive
/gdrive/My Drive
/gdrive/My Drive/geração de moléculas


`Sequencial:` base do modelo 

`LSTM, Embedding, Dense:` tipos de camadas da rede neural

`load_model:` para carregar modelos salvos

`Sequence:` administrador do dataset(q pode n ser usado pois qualquer gerador funciona)




In [ ]:
import pandas as pd
import numpy as np
import re
import random
import math
import ast

from tqdm.auto import tqdm
from google.colab import output

from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Embedding, Dense
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model as load

from tensorflow.keras.utils import Sequence

import matplotlib.pyplot as plt
import seaborn as sns

!pip install kora -q
import kora.install.rdkit

import sys
sys.path.append('notebooks/sascorer')
import sascorer

from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Descriptors
from rdkit.Chem import AllChem
from rdkit import DataStructs


     |████████████████████████████████| 57 kB 3.8 MB/s 
     |████████████████████████████████| 55 kB 4.5 MB/s 


Como obter os arquivos precisos para o SAscore

https://future-chem.com/rdkit-sa-score/#toc5

In [ ]:
!ls

datasets  modelos  moleculas  notebooks


# Carregando e processando o dataset

In [ ]:
a = '/gdrive/MyDrive/geração de moléculas/datasets/CheMBL/chembl.smi'
smiles = []
dir_smi =  'datasets/'+dataset+'/'+dataset.lower()+'.smi'
file = open(dir_smi)
for smi in file:
  smiles.append('^'+smi[:-1]+'$')

dir_dict =  'datasets/'+dataset+'/'+dataset.lower()+'.dict'
f = open(dir_dict)
txt = f.read()
f.close()

dicionario = ast.literal_eval(txt)


input_size = len(dicionario)
output_size = input_size - 1


`Encoder:` transforma  entradas numericas em vetores 

5 -> [0,0,0,0,1,0,0] 

`serializer:` transforma uma entrada para que se adeque à forma de treinamento da RNN, transforma entradas do tipo [1,2,3,4,5,6] em dois arrays do tipo entrada/rotulo

[ 

  [1,2,3],

  [2,3,4],

  [3,4,5]
  
]

[4,5,6]

In [ ]:
def one_hot_encode(dicionario,string):
  #
    linha = []
    symbols = expressao.findall(string)
    for symbol in symbols:
        aux = [0] * len(dicionario) # cria um vetor de zeros do tamanho do dicionario 
        try:
            aux[dicionario[symbol]] = 1 # coloca 1 em 
        except:
            print('\n',symbol, string)
        linha.append(aux)

    return linha

def serializer(vetor, seq_size,smiles):
    X = []
    y = []
    for i in range(len(vetor) - seq_size):
        X.append(  vetor[i:(i+seq_size)]  )
        y.append(  vetor[i+seq_size] )
    return np.array(X),np.array(y)

expressao = re.compile(r'\[.+?\]|.')

alternativa mais simples do gerador

In [ ]:
def gerador(smiles,dicionario,seq_len,shuffle=True,batch_size=1):
    smiles = list(smiles)
    
    if(shuffle):
        random.shuffle(smiles)

    for smile in smiles:
        tamanho = len(expressao.findall(smile))
          
        if (tamanho<=seq_len): 
          #caso as moleculas sejam menores que o tamanho escolhido das janelas no treino da RNN
            n_elementos = 1
            X = np.zeros(  (n_elementos, seq_len, len(dicionario))  )
            y = np.zeros(  (n_elementos, len(dicionario))  )

            vetores = one_hot_encode(dicionario,smile)
            X[0,-(tamanho-1):], y[0] = vetores[:-1], vetores[-1]

        else:
            n_elementos = tamanho - seq_len
            
            vetores = one_hot_encode(dicionario,smile)
            X,y = serializer(vetores,seq_len,smile)
            
        yield X, y[:,:-1]
        
training_generator = gerador(smiles,dicionario,seq_len)


`DataGenerator:` Classe que sobreescrevem funções do tensorflow para processar alguns smiles por vez, treinar o modelo com eles e então descarta-los, evitando assim de exceder a memoria disponivel

Classe chupinhada daqui
https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly

In [ ]:
class DataGenerator(Sequence):

    'Generates data for Keras'
    def __init__(self, smiles, dicionario, seq_len=5, shuffle=True, batch_size=1):
        'Initialization'
        self.smiles = list(smiles)
        self.dicionario = dicionario
        self.seq_len = seq_len
        self.shuffle = shuffle
        self.batch_size = batch_size
        self.on_epoch_end()
        
    def __len__(self):
        'Denotes the number of batches per epoch'
        # return int(np.floor(len(self.list_IDs) / self.batch_size))
        return len(self.smiles)

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.smiles))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        
        # Find list of IDs
        smiles_temp = [self.smiles[k] for k in indexes]
        
        # Generate data
        X, y = self.__data_generation(smiles_temp[0])
        return X, y

    def __data_generation(self, smiles_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        tamanho = len(expressao.findall(smiles_temp))
          
        if (tamanho<=self.seq_len): #caso as moleculas sejam menores que o tamanho escolhido das janelas no treino da RNN
            n_elementos = 1
            X = np.zeros(  (n_elementos, self.seq_len, len(self.dicionario))  )
            y = np.zeros(  (n_elementos, len(self.dicionario))  )


            vetores = one_hot_encode(dicionario,smiles_temp)
            X[0,-(tamanho-1):], y[0] = vetores[:-1], vetores[-1]

        else:
            n_elementos = tamanho - self.seq_len
            # X = np.empty(  (n_elementos, self.seq_len, len(self.dicionario))  )
            # y = np.empty(  (n_elementos, len(self.dicionario))  )
            
            vetores = one_hot_encode(dicionario,smiles_temp)
            X,y = serializer(vetores,self.seq_len,smiles_temp)


        return X, y[:,:-1]

training_generator = DataGenerator(smiles,dicionario,seq_len)

# Criação e treinamento do modelo(~6horas no colab com GPU)  

In [ ]:
dir = 'modelos/RNN' +str(seq_len) + dataset

try:
  vanilla = load(dir)
except:

  vanilla = Sequential()
  # vanilla.add( Embedding( input_size,126) )
  vanilla.add( LSTM(input_size, input_shape=(seq_len,input_size) ) )
  vanilla.add( Dense(126,activation='sigmoid',) )
  vanilla.add( Dense(output_size,activation='softmax'))

  vanilla.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
checkpoint = ModelCheckpoint(
    filepath='modelos/'+modelo,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [ ]:
vanilla.fit(training_generator,
                    # validation_data=validation_generator,
                    use_multiprocessing=True,
                    workers=4,
                    callbacks=[checkpoint],
                    epochs=4)

A saída de streaming foi truncada nas últimas 5000 linhas.
 [Zn] ^CN(C)C(=S)S[Zn]SC(=S)N(C)C$
16567/21412 [======================>.......] - ETA: 57s - loss: 1.5140 - accuracy: 0.5425
 [NH+] ^Nc1ccc(CNC2=[NH+]CCCC2)cc1$
16591/21412 [======================>.......] - ETA: 57s - loss: 1.5137 - accuracy: 0.5426
 [Co++] ^O.O.O.O.O.O.[Cl-].[Cl-].[Co++]$
16667/21412 [======================>.......] - ETA: 56s - loss: 1.5125 - accuracy: 0.5429
 [nH+] ^[NH3+][C@@H](Cc1c[nH]c[nH+]1)C(O)=O$
16712/21412 [======================>.......] - ETA: 55s - loss: 1.5118 - accuracy: 0.5431
 [o+] ^Oc1ccc(cc1)-c1[o+]c2cc(O)cc(O)c2cc1O$
16726/21412 [======================>.......] - ETA: 55s - loss: 1.5115 - accuracy: 0.5431
 [o+] ^[Cl-].COc1cc(ccc1O)-c1[o+]c2cc(O)cc(O)c2cc1O$
16791/21412 [======================>.......] - ETA: 55s - loss: 1.5105 - accuracy: 0.5434
 [o+] ^COc1cc(ccc1O)-c1[o+]c2cc(O)cc(O)c2cc1O[C@@H]1O[C@H](CO)[C@@H](O)[C@H](O)[C@H]1O$
16805/21412 [======================>.......] - ETA: 54s - 

In [ ]:
vanilla.save('modelos/rnn1')

INFO:tensorflow:Assets written to: modelos/rnn1/assets


INFO:tensorflow:Assets written to: modelos/rnn1/assets


# Geração de moléculas

In [ ]:
!ls

datasets  modelos  moleculas  notebooks


In [ ]:
# vanilla = load('vanilaRnn1')
vanilla = load('modelos/'+modelo)

simbolos = list(dicionario.keys())

In [ ]:
novosSmiles = []
for i in tqdm(range (numero_moleculas_geradas)):
    novoCaracter = novoSmile = '^'
    while (novoCaracter !='$'):
        entrada = np.array(one_hot_encode(dicionario, novoSmile))[np.newaxis]
        probabilidades = vanilla.predict(entrada)[0]
        # logprob = [loglog(x) for x in probabilidades ]
        # pred = random.choices(simbolos, logprob)
        pred = random.choices(simbolos, probabilidades)
        novoCaracter = pred[0]
        novoSmile += novoCaracter
    novosSmiles.append(novoSmile)
